In [3]:
import pandas as pd
import os

In [39]:
logs_folder = '/home/abishekthamma/PycharmProjects/masters_thesis/ss-llm/logs'
run_ids = [7761393, 7761405, 7761425, 7761428, 7761445]

def return_val_given_line_str(line_str, split_char="="):
    #usually line of the form "x = y #comment"
    return_val = line_str.split(split_char)[1].split("#")[0].strip().replace(",", "")
    if "*" in return_val:
        return_val = eval(return_val)
    
    return return_val

def return_iter_vals(line_str):
    #usually line of the form "iter 150: loss 6.9318, time 313.88ms, mfu 38.50%
    #print(line_str)
    iter_num = int(line_str.split(":")[0].split(" ")[1])
    loss = float(line_str.split(":")[1].split(",")[0].split(" ")[2])
    time = line_str.split(":")[1].split(",")[1].split(" ")[2].strip()
    mfu = line_str.split(":")[1].split(",")[2].split(" ")[2].strip()
    
    return iter_num, loss, time, mfu

def return_step_vals(line_str):
    #step 250: train loss 6.4297, val loss 6.4395
    step_num = int(line_str.split(":")[0].split(" ")[1])
    train_loss = float(line_str.split(":")[1].split(",")[0].strip().split(" ")[2])
    val_loss = float(line_str.split(":")[1].split(",")[1].strip().split(" ")[2])
    
    return step_num, train_loss, val_loss

def  stats_from_log(log_run_id):
    params_dict = {}
    with open(os.path.join(logs_folder, f'{log_run_id}.out'), 'r') as f:
        lines = f.readlines()
        for line in lines:
            if line[0] == "#":
                continue
            if "batch_size" in line:
                if "batch_size" not in params_dict:
                    params_dict['batch_size'] = int(return_val_given_line_str(line))
            
            if "gradient_accumulation_steps" in line:
                if "gradient_accumulation_steps" not in params_dict:
                    params_dict['gradient_accumulation_steps'] = int(return_val_given_line_str(line))
            
            if "block_size" in line:
                if "block_size" not in params_dict:
                    params_dict['block_size'] = int(return_val_given_line_str(line))
            
            if "tokens per iteration will be" in line:
                if "tokens_per_iteration" not in params_dict:
                    tokens_per_iteration = int(return_val_given_line_str(line, split_char=":"))
                    assert params_dict['block_size']*params_dict['batch_size']*params_dict['gradient_accumulation_steps'] == tokens_per_iteration, "Tokens per iteration mismatch"
                    params_dict['tokens_per_iteration'] = tokens_per_iteration
                    
            if "iter 150" in line:
                iter_num, loss, time, mfu = return_iter_vals(line)
                if "iterations" not in params_dict:
                    params_dict['iteration_num'] = iter_num
                if "loss" not in params_dict:
                    params_dict[f'train_loss_{iter_num}'] = loss
                if "time" not in params_dict:
                    params_dict[f'train_time_{iter_num}'] = time
                if "mfu" not in params_dict:
                    params_dict[f'train_mfu_{iter_num}'] = mfu
            
            if "iter 325" in line:
                iter_num, loss, time, mfu = return_iter_vals(line)
                if "iterations" not in params_dict:
                    params_dict['iteration_num'] = iter_num
                if "loss" not in params_dict:
                    params_dict[f'train_loss_{iter_num}'] = loss
                if "time" not in params_dict:
                    params_dict[f'train_time_{iter_num}'] = time
                if "mfu" not in params_dict:
                    params_dict[f'train_mfu_{iter_num}'] = mfu
            
            if "step 250" in line:
                step_num, train_loss, val_loss = return_step_vals(line)
                if "steps" not in params_dict:
                    params_dict['step_num'] = step_num
                if "train_loss" not in params_dict:
                    params_dict[f'train_loss_{step_num}'] = train_loss
                if "val_loss" not in params_dict:
                    params_dict[f'val_loss_{step_num}'] = val_loss
            
            if "step 500" in line:
                step_num, train_loss, val_loss = return_step_vals(line)
                if "steps" not in params_dict:
                    params_dict['step_num'] = step_num
                if "train_loss" not in params_dict:
                    params_dict[f'train_loss_{step_num}'] = train_loss
                if "val_loss" not in params_dict:
                    params_dict[f'val_loss_{step_num}'] = val_loss
                
            if "CPU Utilized" in line:
                if "cpu_util" not in params_dict:
                    params_dict['cpu_util'] = ":".join(line.split(":")[1:]).strip()
            
            if "CPU Efficiency" in line:
                if "cpu_efficiency" not in params_dict:
                    params_dict['cpu_efficiency'] = ":".join(line.split(":")[1:]).strip()
            
            if "Memory Utilized" in line:
                if "memory_util" not in params_dict:
                    params_dict['memory_util'] = ":".join(line.split(":")[1:]).strip()
            
            if "Memory Efficiency" in line:
                if "memory_efficiency" not in params_dict:
                    params_dict['memory_efficiency'] = line.split(":")[1].strip()
                    
            if "Job Wall-clock time" in line:
                if "wall_clock_time" not in params_dict:
                    params_dict['wall_clock_time'] = ":".join(line.split(":")[1:]).strip()
            
            
    return params_dict

stats_logs = []
for run_id in run_ids:
    stats_logs.append(stats_from_log(run_id))

pd.DataFrame(stats_logs)
            
            
        

,gradient_accumulation_steps,batch_size,block_size,tokens_per_iteration,iteration_num,train_loss_150,train_time_150,train_mfu_150,step_num,train_loss_250,val_loss_250,train_loss_500,val_loss_500,cpu_util,cpu_efficiency,wall_clock_time,memory_util,memory_efficiency
0,8,32,256,65536,150,6.9708,89.76ms,33.65%,500,6.5243,6.5198,6.2471,6.2645,00:12:10,5.79% of 03:30:00 core-walltime,00:02:55,10.96 GB,2.28% of 480.00 GB
1,4,64,256,65536,150,6.9671,83.44ms,35.36%,500,6.5345,6.5344,6.2585,6.2703,00:12:58,6.07% of 03:33:36 core-walltime,00:02:58,10.79 GB,2.25% of 480.00 GB
2,8,64,256,131072,150,6.9517,160.46ms,37.68%,500,6.4695,6.4844,6.1989,6.1946,00:15:31,5.90% of 04:22:48 core-walltime,00:03:39,10.82 GB,2.25% of 480.00 GB
3,16,64,256,262144,150,6.9318,313.88ms,38.50%,500,6.4297,6.4395,6.1489,6.1576,00:00:00,0.00% of 06:02:24 core-walltime,00:05:02,0.00 MB (estimated maximum),0.00% of 480.00 GB (480.00 GB/node)
4,4,128,256,131072,150,6.9378,152.62ms,38.79%,500,6.4711,6.4797,6.2000,6.2094,00:17:25,5.97% of 04:51:36 core-walltime,00:04:03,10.90 GB,2.27% of 480.00 GB


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transformers_version": "4.38.2",
  "use_cache": true,
  "vocab_size": 50257
}